# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [2]:
import pandas as pd

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
rec = pd.read_parquet("recommendations.parquet")
top = pd.read_parquet("top_popular.parquet")
sim = pd.read_parquet("similar.parquet")

In [9]:
top = top.sort_values("users", ascending=False)
top

,item_id,users,avg_track_seq,users_norm
9098,53404,110026,17.389681,1.000000
483876,33311009,101076,225.144396,0.918655
26665,178529,100866,38.155781,0.916746
512157,35505245,95523,155.747914,0.868185
368072,24692821,84153,185.660666,0.764844
...,...,...,...,...
994000,97771067,1,232.000000,0.000000
963411,88842415,1,1236.000000,0.000000
963586,88892071,1,97.000000,0.000000
963611,88892350,1,730.000000,0.000000


In [6]:
events = pd.read_parquet("events.par")


In [7]:
events[events["user_id"] == 10622993]

,user_id,item_id,track_seq,started_at


In [8]:
rec[rec["user_id"] == 10622993]

,user_id,item_id,als_score,artists,genres,albums_limited,user_genre_popularity,user_artist_popularity,cb_score,rank


In [11]:
rec["user_id"].unique()[:100]

array([1000004, 1000005, 1000007, 1000009, 1000011, 1000013, 1000014,
       1000015, 1000019, 1000023, 1000025, 1000026, 1000028, 1000029,
       1000032, 1000037, 1000039, 1000047, 1000060, 1000062, 1000063,
       1000068, 1000072, 1000077, 1000079, 1000080, 1000083, 1000092,
       1000095, 1000101, 1000109, 1000110, 1000115, 1000122, 1000128,
       1000134, 1000135, 1000138, 1000139, 1000143, 1000144, 1000149,
       1000150, 1000154, 1000162, 1000170, 1000171, 1000175, 1000184,
       1000185, 1000189, 1000190, 1000201, 1000203, 1000208, 1000209,
       1000212, 1000215, 1000216, 1000221, 1000227, 1000231, 1000232,
       1000235, 1000243, 1000247, 1000251, 1000254, 1000257, 1000265,
       1000275, 1000277, 1000280, 1000281, 1000286, 1000293, 1000298,
       1000302, 1000304, 1000306, 1000310, 1000311, 1000321, 1000323,
       1000325, 1000331, 1000334, 1000340, 1000343, 1000344, 1000349,
       1000352, 1000360, 1000364, 1000367, 1000369, 1000374, 1000375,
       1000378, 1000

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.